# **Stable Diffusion for Blanka's thesis** 🎨 
*...using `🧨diffusers`*


### Setup

First, please make sure you are using a GPU runtime to run this notebook, so inference is much faster. If the following command fails, use the `Runtime` menu above and select `Change runtime type`.

In [ ]:
!pip install diffusers==0.3.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

You also need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

As google colab has disabled external widgtes, we need to enable it explicitly. Run the following cell to be able to use `notebook_login`

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

Now you can login with your user token.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Stable Diffusion Pipeline


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  

Next, let's move the pipeline to GPU to have faster inference.

In [5]:
pipe = pipe.to("cuda")

In [ ]:
#connecting to anvil
!pip install anvil-uplink
import anvil.server
anvil.server.connect("UPJCXES27WNXQ7VYBWKUWOWQ-2DEO2FXVQQI77LMJ")

In [7]:
#callable function for anvil
from torch import autocast
import anvil.media
import PIL 
@anvil.server.callable
def image_generation(prompt):
  with autocast("cuda"):
    image = pipe(prompt, num_inference_steps=50).images[0]
  image.show()
  image.save("my_image.jpg")
  image_converted = anvil.media.from_file('my_image.jpg')
  return image_converted